### Packages &#x1F4DA;

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_theme()
import pydlm
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from prettytable import PrettyTable
import IPython
import librosa
import librosa.display
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv(r'C:\Users\karol\OneDrive\Área de Trabalho\Mestrado\Códigos\base_completa.csv', sep=';')

### music of index 1,430 &#x1F3B6;	

In [ ]:
df_2 = df[df['file'] == 1430]
features_2 = df_2.iloc[:, 1:-4]

PCA_2 = PCA(n_components = 1, random_state=42)
features_2_PCA = PCA_2.fit_transform(features_2)
features_2_PCA = np.hstack((features_2_PCA, np.ones((58, 1))))

a_2 = df_2['Arousal'].values
v_2 = df_2['Valence'].values

In [ ]:
modelo_dinamico_a2 = pydlm.dlm(a_2) + pydlm.dynamic(features = features_2_PCA, name = 'dynamic', discount = 0.99) 
modelo_dinamico_v2 = pydlm.dlm(v_2) + pydlm.dynamic(features= features_2_PCA, name='dynamics', discount=0.99)
modelo_dinamico_a2.fit()
modelo_dinamico_v2.fit()

In [ ]:
# Filtrar os resultados
modelo_dinamico_a2.fitForwardFilter()

filtered_series = modelo_dinamico_a2.getMean(filterType='forwardFilter')  # (filtered)
smoothed_series = modelo_dinamico_a2.getMean(filterType='backwardSmoother')  # (smoothed)

# Previsão "one-step ahead" para todos os pontos
one_step_predictions = []
for i in range(len(a_2) - 1):  # Prever até o penúltimo ponto
    prediction = modelo_dinamico_a2.predictN(date=i, N=1)[0][0]  # Extrair o valor previsto como escalar
    one_step_predictions.append(prediction)

# Adicionando NaN no último valor para manter o tamanho igual
one_step_predictions.append(float('nan'))

# Extraindo os dados reais para comparação
time_points = range(len(a_2))
observed_data = a_2


plt.figure(figsize=(10, 6)) 
plt.plot(time_points, observed_data, label='Observed', color='black', linestyle='-')
plt.plot(time_points, filtered_series, label='Filtered Series', color='blue',linestyle=':')
plt.plot(time_points, smoothed_series, label='Smoothed Series', color='red', linestyle='-.')
plt.xticks(rotation=45)
plt.tight_layout() 
plt.legend(fontsize=17)
plt.xlabel('Time(s)', fontsize=20)
plt.ylabel('Values', fontsize=20)
plt.ylim(-0.075,0.02)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
#plt.savefig("Arousal.pdf", format = "pdf", bbox_inches="tight")
plt.show()

In [ ]:
# Filtrar os resultados
modelo_dinamico_v2.fitForwardFilter()

filtered_series = modelo_dinamico_v2.getMean(filterType='forwardFilter')  # (filtered)
smoothed_series = modelo_dinamico_v2.getMean(filterType='backwardSmoother')  # (smoothed)

# Previsão "one-step ahead" para todos os pontos
one_step_predictions = []
for i in range(len(v_2) - 1):  # Prever até o penúltimo ponto
    prediction = modelo_dinamico_v2.predictN(date=i, N=1)[0][0]  # Extrair o valor previsto como escalar
    one_step_predictions.append(prediction)

# Adicionando NaN no último valor para manter o tamanho igual
one_step_predictions.append(float('nan'))

# Extraindo os dados reais para comparação
time_points = range(len(v_2))
observed_data = v_2

plt.figure(figsize=(10, 6))
plt.plot(time_points, observed_data, label='Observed', color='black', linestyle='-')
plt.plot(time_points, filtered_series, label='Filtered Series', color='blue', linestyle=':')
plt.plot(time_points, smoothed_series, label='Smoothed Series', color='red', linestyle='-.')
plt.xticks(rotation=45) 
plt.tight_layout()  
plt.legend(fontsize=17)  
plt.xlabel('Time(s)', fontsize=20)
plt.ylabel('Values', fontsize=20)
plt.xticks(fontsize=20)  
plt.yticks(fontsize=20) 
#plt.savefig("Valence.pdf", format = "pdf", bbox_inches="tight")
plt.show()

### Metrics &#x1F523;	

In [ ]:
inter = 0
most_important_name = []
least_important_name = []
most_important_value = []
least_important_value = []
variance_ratio = []
top_n = 13
salient = pd.DataFrame(columns=[f'Feature_{i+1}' for i in range(top_n)] + [f'Valor_{i+1}' for i in range(top_n)])

filtered_series_a = []  # Séries filtradas para Arousal
smoothed_series_a = []  # Séries suavizadas para Arousal
filtered_series_v = []  # Séries filtradas para Valence
smoothed_series_v = []  # Séries suavizadas para Valence


for i in df['file'].unique():
    inter +=1
    df_i = df[df['file'] == i] # cria um dataframe para cada música
    features_i = df_i.iloc[:, 1:-4] # retira columna = [Frametime, Arousal, Valence, File]
    PCA_i = PCA(n_components = 1, random_state=42) # cria um PCA
    features_i_PCA = PCA_i.fit_transform(features_i) # treina a partir das features
    features_i_PCA = np.hstack((features_i_PCA, np.ones((len(df_i), 1)))) # adiciona um ''interceptor''
    loadings = PCA_i.components_[0] # pega os coefs da primeira componente
    sorted_indices = np.argsort(np.abs(loadings))[::-1]  # Ordena em ordem decrescente de magnitude
    top_features_indices = sorted_indices[:top_n]  # Pega os índices das top features
    top_features_names = df_i.columns[top_features_indices] # pega os nomes
    top_features_values = loadings[top_features_indices] # pega os valores
    row = list(top_features_names) + list(top_features_values)
    salient.loc[i] = row

    a_i = df_i['Arousal'].values # arousal
    v_i = df_i['Valence'].values # valence
    modelo_dinamico_ai = pydlm.dlm(a_i) + pydlm.dynamic(features = features_i_PCA, name = 'dynamic', discount = 0.99) 
    modelo_dinamico_vi = pydlm.dlm(v_i) + pydlm.dynamic(features= features_i_PCA, name='dynamics', discount=0.99)
    modelo_dinamico_ai.fit()
    modelo_dinamico_vi.fit()

    filtered_series_ai = modelo_dinamico_ai.getMean(filterType='forwardFilter')  # Séries filtradas (filtered)
    smoothed_series_ai = modelo_dinamico_ai.getMean(filterType='backwardSmoother')  # Séries suavizadas (smoothed)
    filtered_series_vi = modelo_dinamico_ai.getMean(filterType='forwardFilter')  # Séries filtradas (filtered)
    smoothed_series_vi = modelo_dinamico_ai.getMean(filterType='backwardSmoother')  # Séries suavizadas (smoothed)
    
    # Calculando MSE para a série filtrada
    filtered_series_ai = mean_squared_error(a_i, filtered_series_ai)
    filtered_series_a.append(filtered_series_ai)
    filtered_series_vi = mean_squared_error(v_i, filtered_series_vi)
    filtered_series_v.append(filtered_series_vi)
    
    # Calculando MSE para a série suavizada
    smoothed_series_ai = mean_squared_error(a_i, smoothed_series_ai)
    smoothed_series_a.append(smoothed_series_ai)
    smoothed_series_vi = mean_squared_error(v_i, smoothed_series_vi)
    smoothed_series_v.append(smoothed_series_vi)
    

    #mse_ai = modelo_dinamico_ai.getMSE()
    #mse_a.append(mse_ai)
    #mse_vi = modelo_dinamico_vi.getMSE()
    #mse_v.append(mse_vi)    

    most_importanti = np.abs(pd.DataFrame(PCA_i.components_,columns=features_i.columns))
    most_important_name.append(most_importanti.max().idxmax())
    most_important_value.append(most_importanti.max(axis=1))
#    least_importanti = pd.DataFrame(PCA_i.components_,columns=features_i.columns) 
    least_important_name.append(most_importanti.min().idxmin())
    least_important_value.append(most_importanti.min(axis=1))
    variance_ratio.append(PCA_i.explained_variance_ratio_)

In [ ]:
filtered_series_a = pd.Series(filtered_series_a)
filtered_series_v = pd.Series(filtered_series_v)
smoothed_series_a = pd.Series(smoothed_series_a)
smoothed_series_v = pd.Series(smoothed_series_v)

rmse_filtered_a = np.sqrt(filtered_series_a.mean()).round(3)
rmse_filtered_v = np.sqrt(filtered_series_v.mean()).round(3)
rmse_smoothed_a = np.sqrt(smoothed_series_a.mean()).round(3)
rmse_smoothed_v = np.sqrt(smoothed_series_v.mean()).round(3)

table = PrettyTable()
table.field_names = ["Serie", "Arousal", "Valence"]
table.add_row(["Filtered", rmse_filtered_a, rmse_filtered_v])
table.add_row(["Smoothed", rmse_smoothed_a, rmse_smoothed_v])

print(table)

### PCA &#x1F4C9;		

In [ ]:
important_np = np.array(most_important_value)

# Calcular a porcentagem de músicas
porcentagem = np.mean(important_np >= 0.80) * 100

print(f"A porcentagem de músicas onde uma única componente explica mais de 80% da variabilidade é: {porcentagem:.2f}%")

In [ ]:
threshold = 0.95

# Somar as colunas Valor_1, Valor_2 e Valor_3
sum_values = salient['Valor_1'] + salient['Valor_2'] + salient['Valor_3']

# Contar quantas músicas têm a soma de Valor_1 + Valor_2 + Valor_3 maior ou igual a 95%
count_above_threshold = salient[sum_values >= threshold].shape[0]

# Calcular o total de músicas (linhas no dataframe)
total_musics = salient.shape[0]

# Calcular a porcentagem de músicas que atendem a condição
percentage_above_threshold = (count_above_threshold / total_musics) * 100

print(f"Porcentagem de músicas com Valor_1 + Valor_2 + Valor_3 >= 95%: {percentage_above_threshold:.2f}%")